In [1]:
import gym
import torch
import random
import torchvision.transforms as T
from PIL import Image
env = gym.envs.make("PongDeterministic-v4")
from matplotlib.pyplot import imshow
import numpy as np

#try:


In [2]:
import time
from time import sleep

In [3]:
n_action = 3


In [4]:
from collections import deque
import copy
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

In [5]:
class CNNModel(nn.Module):
    def __init__(self, n_channel, n_action):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=n_channel, out_channels=32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 32, 4, stride=2)
        self.conv3 = nn.Conv2d(32, 32, 3, stride=1)
        self.fc = torch.nn.Linear(22 * 16 * 32, 512)
        self.out = torch.nn.Linear(512, n_action)
    def forward(self, x):
#         print(x.shape)
        x = F.relu(self.conv1(x))
#         print(x.shape)
        x = F.relu(self.conv2(x))
#         print(x.shape)
        x = F.relu(self.conv3(x))
#         print(x.shape)
        x = x.reshape(x.size(0), -1)
#         print(x.shape)
        x = F.relu(self.fc(x))
#         print(x.shape)
        output = self.out(x)
        return output

In [6]:
class CNNModel(nn.Module):
    def __init__(self, n_channel, n_action):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=n_channel, out_channels=32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, 3, stride=1)
        self.fc = torch.nn.Linear(7 * 7 * 64, 512)
        self.out = torch.nn.Linear(512, n_action)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc(x))
        output = self.out(x)
        return output

In [7]:
class DQN():
    def __init__(self, n_channel, n_action, lr=0.05):
        self.criterion = torch.nn.MSELoss()
        self.model = CNNModel(n_channel, n_action)
        self.model.to(device)
        self.model_target = copy.deepcopy(self.model)
        self.model.share_memory()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)
        print(self.model)
#         for n. p in self.model.named_parametrs():
#             print(n,p.device)
    def update(self, s, y):
        """
        Update the weights of the DQN given a training sample
        @param s: state
        @param y: target value
        """
        #print("y_pred loss",np.array(s).shape, np.array(y).shape)
        y_pred = self.model(torch.tensor(s, dtype = torch.float32).to(device))
        loss = self.criterion(y_pred, torch.tensor(y, dtype = torch.float32).to(device))
       

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    def predict(self, s):
        """
        Compute the Q values of the state for all actions using the learning model
        @param s: input state
        @return: Q values of the state for all actions
        """
        with torch.no_grad():
            return self.model(torch.Tensor(s, dtype = torch.float32))#.to(device))
    def target_predict(self, s):
        """
        Compute the Q values of the state for all actions using the target network
        @param s: input state
        @return: targeted Q values of the state for all actions
        """
        with torch.no_grad():
            return self.model_target(torch.Tensor(s, dtype = torch.float32).to(device))#.to(device))

    def replay(self, memory, replay_size, gamma):
        """
        Experience replay with target network
        @param memory: a list of experience
        @param replay_size: the number of samples we use to update the model each time
        @param gamma: the discount factor
        """
        if len(memory) >= replay_size:
            replay_data = random.sample(memory, replay_size)
            states = []
            td_targets = []
            for state, action, next_state, reward, is_done in replay_data:
                states.append(state.tolist()[0])
                q_values = self.predict(state).tolist()[0]
                if is_done:
                    q_values[action] = reward
                else:
                    q_values_next = self.target_predict(next_state).detach()

                    q_values[action] = reward + gamma * torch.max(q_values_next).item()

                td_targets.append(q_values)

            self.update(states, td_targets)

    def copy_target(self):
        self.model_target.load_state_dict(self.model.state_dict())

In [8]:
class DQN():
    def __init__(self, n_channel, n_action, lr=0.05):
        self.criterion = torch.nn.MSELoss()
        self.model = CNNModel(n_channel, n_action)
        self.model.to(device)
       # self.model_target = copy.deepcopy(self.model)
        self.model_target = CNNModel(n_channel, n_action)
        self.model_target.to(device)   
        self.model.share_memory()
        self.model_target.load_state_dict(self.model.state_dict()) 
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)
        print(self.model)
        print("trainable_params",sum(p.numel() for p in self.model.parameters() if p.requires_grad))

    def update(self, s, y):
        """
        Update the weights of the DQN given a training sample
        @param s: state
        @param y: target value
        """
        #print("y_pred loss",np.array(s).shape, np.array(y).shape)
        y_pred = self.model(torch.tensor(s, dtype = torch.float32).to(device))
        loss = self.criterion(y_pred, torch.tensor(y, dtype = torch.float32).to(device))
       

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    def predict(self, s):
        """
        Compute the Q values of the state for all actions using the learning model
        @param s: input state
        @return: Q values of the state for all actions
        """
       # with torch.no_grad():
        self.model.share_memory()
        return self.model(torch.tensor(s, dtype = torch.float32).to(device))#)
    def target_predict(self, s):
        """
        Compute the Q values of the state for all actions using the target network
        @param s: input state
        @return: targeted Q values of the state for all actions
        """
        with torch.no_grad():
            return self.model_target(torch.tensor(s, dtype = torch.float32).to(device))#.to(device))

    def replay(self, memory, replay_size, gamma):
        """
        Experience replay with target network
        @param memory: a list of experience
        @param replay_size: the number of samples we use to update the model each time
        @param gamma: the discount factor
        """
        
        if len(memory) >= replay_size:
            s1 = time.perf_counter()
            replay_data = random.sample(memory, replay_size)
            s2 = time.perf_counter()
           # time1.append(s2-s1)
            #states = []
            states = np.zeros((replay_size,3, 84, 84),dtype = "float32")
            td_targets = []
            i = 0
            for state, action, next_state, reward, is_done in replay_data:
                
              #  print(state, state.shape, type(state))
                s1 = time.perf_counter()
                states[i]= state.numpy()[0]
#                 states.append(state.tolist()[0])
                s2 = time.perf_counter()
                time2.append(s2-s1)   #!!!
                s1 = time.perf_counter()
                q_values = self.predict(state).tolist()[0]
                #print(q_values)
                s2 = time.perf_counter()
               # time3.append(s2-s1)   #!!!
                if is_done:
                    q_values[action] = reward
                else:
                    s1 = time.perf_counter()
                    q_values_next = self.target_predict(next_state).detach()
                   # print(111, self.target_predict(next_state))
                    s2 = time.perf_counter()
                  #  time4.append(s2-s1)    #!!!
                    s1 = time.perf_counter()
                    q_values[action] = reward + gamma * torch.max(q_values_next).item()
                    s2 = time.perf_counter()
                  #  time5.append(s2-s1)
                td_targets.append(q_values)
                i+=1
            s1 = time.perf_counter()
            self.update(states, td_targets)
            s2 = time.perf_counter()
           # time6.append(s2-s1)
#         print("time1",np.mean(np.array(time1)),np.sum(np.array(time1)), len(time1))
#             print("time2",np.mean(np.array(time2)), np.sum(np.array(time2)),len(time2))
#         print("time3",np.mean(np.array(time3)),np.sum(np.array(time3)), len(time3))
#         print("time4",np.mean(np.array(time4)), np.sum(np.array(time4)),len(time4))
#         print("time5",np.mean(np.array(time5)),np.sum(np.array(time5)), len(time5))
#         print("time6",np.mean(np.array(time6)), np.sum(np.array(time6)),len(time6))

    def copy_target(self):
        self.model_target.load_state_dict(self.model.state_dict())

In [9]:
device = torch.device('cpu')

In [10]:
torch_model = CNNModel(3, 3)

In [11]:
n_episode = 1000
lr = 0.00025
replay_size = 32
target_update = 10
dqn = DQN(3, n_action, lr)
memory = deque(maxlen=100000)
total_reward_episode = [0] * n_episode

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc): Linear(in_features=3136, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=3, bias=True)
)
trainable_params 1683619


In [12]:
pytorch_total_params = sum(p.numel() for p in torch_model.parameters() if p.requires_grad)
pytorch_total_params

1683619

In [13]:
x = torch.tensor(np.random.rand(32, 3, 84, 84),dtype = torch.float32)#.shape
y = torch.tensor(np.random.rand(32, 3),dtype = torch.float32)

In [14]:
#x = np.random.rand(3).astype("float32")

In [15]:
x = np.random.rand(1, 3, 84, 84).astype("float32")#.shape
x.dtype

dtype('float32')

In [16]:
# %%time
# import torch.multiprocessing as mp


# def train(model):
#     # Construct data_loader, optimizer, etc.
#     for data, labels in (x, y):
#         optimizer.zero_grad()
#         loss_fn(model(data), labels).backward()
#         optimizer.step()  # This will update the shared parameters

# #if __name__ == '__main__':
# num_processes = 1
# model = CNNModel(3, 3).to(device)
# # NOTE: this is required for the ``fork`` method to work
# #model.share_memory()
# processes = []
# for rank in range(num_processes):
#     #sleep(2)
#     p = mp.Process(target=train, args=(model,))
#     p.start()
#     processes.append(p)
# for p in processes:
#     p.join()

In [17]:
%%time
aa = dqn.predict(x)

CPU times: user 39.6 ms, sys: 0 ns, total: 39.6 ms
Wall time: 4.95 ms


In [18]:
%%time
aa = dqn.predict(x)

CPU times: user 15.6 ms, sys: 9.81 ms, total: 25.4 ms
Wall time: 2.09 ms


In [19]:
%timeit    dqn.predict(x)

811 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
import time

In [21]:
s1 = time.perf_counter()
kk = dqn.predict(x)
s2 = time.perf_counter()



In [22]:
print(round(s2-s1, 6),"/n")

0.001771 /n


In [23]:
# from torch.multiprocessing import Pool, Process, set_start_method
# #try:
# set_start_method('spawn')
# #except RuntimeError:
#    pass


In [24]:
# %%time

# def f(x):
#     time.sleep(3)
#     return x*x


# with Pool(5) as p:
#     print(p.map(f, [1, 2, 3]))
#     print(p.map(f, [1, 2, 3]))
#     print(p.map(f, [1, 2, 3]))


In [26]:
x.shape

(1, 3, 84, 84)

In [27]:
from torch.multiprocessing import Pool, Process, set_start_method
#set_start_method('spawn')
#orch.multiprocessing.get_all_sharing_strategies()

In [28]:
torch_model = CNNModel(3, 3)

In [29]:
torch_model.to(device)

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc): Linear(in_features=3136, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=3, bias=True)
)

In [30]:
torch_model(torch.tensor(x, dtype = torch.float32).to(device))

tensor([[ 0.0319,  0.0105, -0.0247]], grad_fn=<AddmmBackward>)

In [35]:
import numpy as np
import time
listx = []
for i in range(32):
    listx.append(torch.tensor(x, dtype = torch.float32) )  

In [36]:
#from multiprocessing import Pool
if __name__ == '__main__':
    with Pool(3) as p:
        p.map(torch_model, listx)



MaybeEncodingError: Error sending result: '[tensor([[ 0.0319,  0.0105, -0.0247]], grad_fn=<AddmmBackward>), tensor([[ 0.0319,  0.0105, -0.0247]], grad_fn=<AddmmBackward>), tensor([[ 0.0319,  0.0105, -0.0247]], grad_fn=<AddmmBackward>)]'. Reason: 'RuntimeError('Cowardly refusing to serialize non-leaf tensor which requires_grad, since autograd does not support crossing process boundaries.  If you just want to transfer the data, call detach() on the tensor before serializing (e.g., putting it on the queue).')'

In [33]:
def predict_test(x):
    torch_model()
#     s1 = time.perf_counter()
#     kk = dqn.predict(x)
#     s2 = time.perf_counter()
#     print(round(s2-s1, 5),"/n")
    return round(s2-s1, 5)

In [20]:
predict_test(x)

TypeError: forward() missing 1 required positional argument: 'x'

In [28]:
import numpy as np
import time
listx = []
for i in range(32):
    listx.append(np.random.rand(1, 3, 84, 84).astype("float32"))   

In [ ]:
import concurrent.futures as pool

In [ ]:
executer = pool.ProcessPoolExecutor(max_workers=1)

In [ ]:
executerlist = list()

In [ ]:
for name in listx:
    executerlist.append(executer.submit(predict_test,
                                        x = name,
#                                         yy = yy,
#                                         zz = zz,
                                        ))
    

In [ ]:
ok = pool.wait(executerlist)

In [ ]:
summ = 0 
for future in ok.done:
    result = future.result()
    summ += result
   # print(result)
print(1111,summ)
del executerlist

In [ ]:
1111 0.051899999999999995

In [ ]:
try:
    executer.shutdown()
except: pass


In [ ]:
x = torch.tensor(x)
x.dtype

In [ ]:
%timeit x.detach()

In [ ]:
#k = []
k = np.zeros((32,3, 84, 84),dtype = "float32")

In [ ]:
k[0].shape

In [ ]:
%%time
for i in range (32):
    k[i]= (x.numpy()[0])

In [ ]:
k[0]

In [ ]:
k[0].shape

In [ ]:
%timeit torch.tensor(k, dtype = torch.float32)

In [ ]:
%%time
for i in range(100):
    k.append(x.tolist()[0])

In [ ]:
del k

In [ ]:
%timeit k.append((x.numpy()[0]))

In [ ]:
%timeit k.append(x.tolist()[0])

In [ ]:
d = x.tolist()

In [ ]:
%timeit  k.append(d)

In [ ]:
%timeit x.tolist()

In [ ]:
%timeit x.tolist()

In [ ]:
%timeit k.append(x.numpy()[0])

In [ ]:
%%time
for i in range(10000):
    kk = x.tolist()

In [ ]:
k

In [ ]:
len(k)

In [ ]:
#dqn.update(x,y)

In [ ]:
%%time
for i in range(10):
    dqn.update(x,y)

In [ ]:
cuda
first
Wall time: 419 ms
    
cpu
Wall time: 210 ms

In [ ]:
cuda
second
Wall time: 64.4 ms
cpu    
Wall time: 162 ms

In [ ]:
batch 2048

In [ ]:
cuda
first
Wall time: 980 ms
    
cpu
Wall time: 3.51 s

In [ ]:
cuda
second
Wall time: 627 ms
cpu    
Wall time: 3.27 s

In [ ]:
second 2048
keras      torch
    cpu:
3.31 s     3.27 s  
    cuda:
880 ms    627 ms

# KERAS

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

In [ ]:
model_keras = keras.models.Sequential()
model_keras.add(keras.layers.Conv2D(32, (8,8), input_shape=( 210, 160, 3), strides=(4, 4)))
model_keras.add(keras.layers.ReLU())
# model_keras.add(keras.layers.BatchNormalization())
model_keras.add(keras.layers.Conv2D(32, (4,4),strides=(2, 2)))
model_keras.add(keras.layers.ReLU())
# model_keras.add(keras.layers.BatchNormalization())
model_keras.add(keras.layers.Conv2D(32, 3, 1))
model_keras.add(keras.layers.ReLU())
# model_keras.add(keras.layers.ELU())
# model_keras.add(keras.layers.BatchNormalization())
# model_keras.add(keras.layers.MaxPool2D(pool_size=(2, 2), strides= (2,2)))
model_keras.add(keras.layers.Flatten())
model_keras.add(keras.layers.Dense(512))
model_keras.add(keras.layers.ReLU())
model_keras.add(keras.layers.Dense(3))
model_keras.add(keras.layers.ReLU())
model_keras.compile(loss='mse', optimizer='Adam')

In [ ]:
model_keras.summary()

In [ ]:
x = np.random.rand(32,  210, 160, 3)#.shape
y = np.random.rand(32, 3)



In [ ]:
%%time
model_keras.fit(x,y, epochs = 10, batch_size = 32, verbose = 0)

In [ ]:
# cpu
# first 
# Wall time: 580 ms
# second
# Wall time: 166 ms

In [ ]:
# cuda
# first 
#Wall time: 17.1 s
#second   
#Wall time: 93 ms
    

In [ ]:
#batch 2048

In [ ]:
# cpu
# first 
# Wall time: 3.96 s
# second
# Wall time: 3.31 s

In [ ]:
# cuda
# first 
#Wall time: 5.26 s

#second   
#WWall time: 880 ms
    

In [ ]:
cuda
torch:
    